<a href="https://colab.research.google.com/github/cocomarine25/LBnC/blob/main/finalterm_study/class2023Fall_1110.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install MySQL Server
!apt-get update
!apt-get install mysql-server
# Install MySQL python deps
!pip install mysql-connector-python
# Install FastAPI python deps
!pip install fastapi[all]
!pip install pyngrok
!pip install uvicorn

# Start the MySQL service
!service mysql start

# Log in to MySQL as root
!mysql -u root


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,282 kB]
Get:13 http://archive.ubuntu.com/ubuntu

```mysql
DROP USER IF EXISTS 'test'@'localhost';
CREATE USER 'test'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY '1234';
GRANT ALL PRIVILEGES ON *.* TO 'test'@'localhost' WITH GRANT OPTION;
FLUSH PRIVILEGES;

DROP DATABASE IF EXISTS TestDataset;
CREATE DATABASE TestDataset;
CREATE TABLE TestDataset.TestTable (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(255),
    Age INT
);
INSERT INTO TestDataset.TestTable (Name, Age) VALUES
('John', 30),
('Alice', 25),
('Bob', 35);
exit;
```

In [3]:
# Import necessary libraries
from fastapi import FastAPI, Form, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
import mysql.connector
from pyngrok import ngrok
import time
import uvicorn
from multiprocessing import Process, Queue

# Create a Queue for inter-process communication
log_queue = Queue()

# Set up ngrok tunnel to forward traffic to port 8000 of google colab device
ngrok.set_auth_token('2XebBqex181SuV0M5Nyxo2Ec0ot_7qoYZbX9egm67Dw9XHcBj')
public_url = ngrok.connect(addr="8000")
print(f"Public URL: {public_url}")

# Create a FastAPI app
app = FastAPI()

# Function to establish a database connection
def connect_to_database():
    return mysql.connector.connect(
        host="localhost",  # Database server address
        user="test",       # Database username
        password="1234",   # User's password
        database="TestDataset"  # Name of the database
    )

# Function to fetch all records from the database
# 우리가 만들었던거 가져와서 실행,
def fetch_all_records():
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "SELECT * FROM TestDataset.TestTable"
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        return []

# Handle get request to /
@app.get("/", response_class=HTMLResponse)
def read_root(name: str = None):
    records = fetch_all_records()
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>My Form</title>
        <style>
            /* Add your CSS styling here */
            body { font-family: Arial, sans-serif; color: blue; }
            input { margin: 10px; }
            .highlighted { color: red; }  /* CSS class for highlighting */ <!--read 할 때 빨간색으로 나오게 하는것-->
        </style>
    </head>
    <body>
        <h1>Last time...</h1>
        <form action="/submit" method="post" accept-charset="UTF-8">
            <input type="text" name="input_field" placeholder="Enter something">
            <input type="submit" value="Submit">
        </form>
        <h1>CRUD Operations</h1>
        <form action="/create" method="post" accept-charset="UTF-8">
            <input type="text" name="name" placeholder="Name">
            <input type="number" name="age" placeholder="Age">
            <input type="submit" value="Create">
        </form>
        <form action="/read" method="get">
            <input type="text" name="name" placeholder="Search by Name">
            <input type="submit" value="Read">
        </form>
        <form action="/update" method="post" accept-charset="UTF-8">
            <input type="text" name="old_name" placeholder="Old Name">
            <input type="text" name="new_name" placeholder="New Name">
            <input type="number" name="new_age" placeholder="New Age">
            <input type="submit" value="Update">
        </form>
        <form action="/delete" method="post" accept-charset="UTF-8">
            <input type="text" name="name" placeholder="Name to Delete">
            <input type="submit" value="Delete">
        </form>
        <h2>Database Records</h2>
        <ul> <!-- (unorder list)list를 open한 것, 잠시 잘라두고 아래 스트링을 추가한 후에 닫는다-->
    """

    for record in records:
        if name and record[1] == name:
            html_content += f'<li class="highlighted">ID: {record[0]}, Name: {record[1]}, Age: {record[2]}</li>'
        else:
            html_content += f'<li>ID: {record[0]}, Name: {record[1]}, Age: {record[2]}</li>'

    html_content += """
        </ul>
        <script>
            // Add your JavaScript here if needed
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content, media_type="text/html; charset=utf-8")


# Handle post request to /submit
@app.post("/submit")
async def submit(input_field: str = Form(...)):
    try:
        message = f"input_field: {input_field}"
        log_queue.put(message)  # Put the log message into the queue
        return JSONResponse(content={"message": message}, status_code=200)
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Create a new record
@app.post("/create")
def create_record(name: str = Form(...), age: int = Form(...)):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "INSERT INTO TestDataset.TestTable (Name, Age) VALUES (%s, %s)" # TestDataset.TestTable에 name, age에 삽입을 할거고, %s, %s는 스트링, 스트링을 삽입한다는 의미
        cursor.execute(query, (name, age))
        conn.commit() # 해라
        conn.close() # 열었던 것은 닫아라
        return read_root()  # Re-render
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Retrieve data
@app.get("/read")
def read_record(name: str): # 이해하기 편하게 name을 썼지만 실제로 name이라는 변수명을 사용하는 것은 위험하다
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "SELECT * FROM TestDataset.TestTable WHERE Name = %s"
        cursor.execute(query, (name,))
        result = cursor.fetchall()
        conn.close()
        return read_root(result[0][1])  # Update   # read_root 대신 위에 name을 써도 된다   # 내용물이 없다면 에러가 나서 화면이 바뀌지 않는다 (빨간 글씨로 표시되지 않음)
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Update existing data
@app.post("/update")
def update_record(old_name: str = Form(...), new_name: str = Form(...), new_age: int = Form(...)):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "UPDATE TestDataset.TestTable SET Name = %s, Age = %s WHERE Name = %s"
        cursor.execute(query, (new_name, new_age, old_name))
        conn.commit()
        conn.close()
        return read_root()  # Re-render
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Delete data
@app.post("/delete")
def delete_record(name: str = Form(...)):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "DELETE FROM TestDataset.TestTable WHERE Name = %s"
        cursor.execute(query, (name,))
        conn.commit()
        conn.close()
        return read_root()  # Re-render
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Start FastAPI
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000) # host에는 내부 ip, 외부 ip, 루프백을 쓸 수 있음. 0.0.0.0은 루프백이다.(내 컴퓨터에 키는 것)   # port는 내 프로그램이 돌아가고 있는 주소값

# Function to print log messages from the Queue
# 아무것도 안하고 프린트만 하는 것. 멀티프로세스 때문에 사용
def print_log_messages(queue):
    while True:
        message = queue.get()  # Get the log message from the queue
        print(message)

# Using multiprocessing
run_process = Process(target=run)
run_process.start()
print_process = Process(target=print_log_messages, args=(log_queue,))
print_process.start()

# Give the server a moment to start
time.sleep(3)

# Block
# 이게 없으면 다음 코드가 실행되고, 프로그램이 종료됨.
# 무엇인가를 입력하게 되면 다음 코드가 실행되면서 종료됨.
blocking_main_proc = input()

# Kill
run_process.terminate()
print_process.terminate()

# Double check
!kill $(pgrep ngrok)


Public URL: NgrokTunnel: "https://3a7a-34-80-102-251.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2308]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     163.152.3.180:0 - "GET / HTTP/1.1" 200 OK
INFO:     163.152.3.180:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     163.152.3.180:0 - "POST /submit HTTP/1.1" 200 OK
input_field: 집가고싶다
INFO:     163.152.3.180:0 - "POST /create HTTP/1.1" 200 OK
INFO:     163.152.3.180:0 - "GET /read?name=%EC%9C%A4%ED%83%9C%EB%AF%BC HTTP/1.1" 200 OK
INFO:     163.152.3.180:0 - "POST /delete HTTP/1.1" 200 OK
INFO:     163.152.3.180:0 - "GET / HTTP/1.1" 200 OK
INFO:     163.152.3.180:0 - "POST /submit HTTP/1.1" 200 OK
input_field: 김시헌
INFO:     163.152.3.180:0 - "POST /create HTTP/1.1" 200 OK
INFO:     163.152.3.180:0 - "GET /read?name=%EC%9C%A4%ED%83%9C%EB%AF%BC HTTP/1.1" 200 OK
끝


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2308]


In [ ]:
# 총 5개의 프로세스가 돌아가고 있음.

# 가장 처음에 (DB),
# 파이썬에서 (print), (웹서버), (구글 코랩),
# (내 컴퓨터)



In [ ]:
# 위에서 name, age에 글씨를 입력하고 엔터를 누르면 화면이 위로 올라가면서 사람이 추가가 되는데,
# 이것은 화면이 전체적으로 렌더링 되면서 이루어지는 것이다.
# 프로같은 코딩은 전체적으로 렌더링 되는 것이 아니라 대가 보고있는 화면 그대로 글씨만 한 줄 추가되는 것이다.